In [2]:
import pennylane as qml
from pennylane import numpy as np
from tensorflow import keras
import random

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
import dill

import utils_fs
import utils_kernel
import utils_circuit

In [3]:
#可运行mRMR算法和random固定特征的数据+多进程
n_train = 100    # Size of the train dataset
n_test = 60     # Size of the test dataset
n_dim = 8       # 需要降到多少维
target_label_list = [0,1,2] #需要提取数据的标签列表
run_numbers = 50          #重复运行多少次,每次随机设置gate-encoding 线路      

SAVE_PATH = "data/" # Data saving folder
PREPROCESS = True          # If False, skip processing and load data from SAVE_PATH
select_samples_with_labels = True       # 是否挑选特定标签的数据
FS_state = True              # 是否进行图像特征提取
fs_type = 'random'               #图像特征提取的方式，'LBP','HOG','random','mRMR'
load_selected_index = False   # 是否加载随机选择或者mRMR特征的索引
mnist_dataset = keras.datasets.mnist  
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

n_class = len(target_label_list)

In [4]:
if select_samples_with_labels == True:
    test_images, test_labels = utils_fs.extract_data_with_label(test_images, test_labels, target_label_list)
    train_images, train_labels = utils_fs.extract_data_with_label(train_images, train_labels, target_label_list)  

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]


def feature_selection(images, labels, fs_type, selected_index_list):
    processed_images = []
    if fs_type == 'LBP':
        processed_images = utils_fs.fs_with_LBP(images, 1)
    elif fs_type == 'HOG':
        processed_images = utils_fs.fs_with_HOG(images)
    elif fs_type == 'random':
        #random_index_list = get_random_selected_list(n_dim)
        processed_images = utils_fs.fs_with_random(images, selected_index_list)
    elif fs_type == 'mRMR':
        #mRMR_index_list = get_mRMR_selected_list_with_train_data(train_images, train_labels, n_dim)
        processed_images = utils_fs.fs_with_mRMR(images, labels, selected_index_list)

    return processed_images 


if FS_state == True:
    selected_index_list = []
    if fs_type == 'mRMR':
        selected_index_list = utils_fs.get_mRMR_selected_list_with_train_data(n_dim, train_images, train_labels, 'selected_feature_index/selected_feature_index_list_4_mRMR_cls3_0925_50_1.npy')
    elif fs_type == 'random':
        selected_index_list = utils_fs.get_random_selected_list(n_dim, 784, 'selected_feature_index/sfIndex_cls3_random_0925_50_4.npy')

    test_images = feature_selection(test_images, test_labels, fs_type, selected_index_list)
    train_images = feature_selection(train_images, train_labels, fs_type, selected_index_list)

In [5]:
# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

#当维数特别低时候，比如少于20维时，有些数据就全为0，需要剔除,label要同步处理
train_images, train_labels = utils_fs.remove_invalid_image_and_labels(train_images, train_labels)
test_images, test_labels = utils_fs.remove_invalid_image_and_labels(test_images, test_labels)

In [6]:
def process_img_to_features(image):
    img = image.flatten()
    return img

if PREPROCESS == True: #将图像数据拉成1维
    new_train_images = []
    print("pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, np.shape(train_images)[0]), end="\r")
        new_train_images.append(process_img_to_features(img))
      
    train_images = np.array(new_train_images, requires_grad=False)

    new_test_images = []
    print("\npre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, np.shape(test_images)[0]), end="\r")
        new_test_images.append(process_img_to_features(img))
   
    test_images = np.array(new_test_images, requires_grad=False)

pre-processing of train images:
89/89        
pre-processing of test images:


In [7]:
n_qubits = n_dim

dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

#随机生成100种路线，8根线路，RX，RY，RY就有3**8，乘以 H门有2**8 乘以CNOT门有2**8，一共有429981696
#用列表代表线路门的排列：一共3层，第一层0代表RX门，1代表RY门，2代表RZ门；第二层1代表使用H门，0代表不使用，第三层0代表不用CNOT门，1代表设置CNOT门，
#且与下一个qubit链接，比如目前在第1个qubit，则CNOT门是链接第1和第2qubit。

def generate_circuit_code(wires):
    circuit_code = []
    for i in range(3):
        for j in range(wires):
            if(i == 0):
                circuit_code.append(random.randint(0,2))
            else:
                circuit_code.append(random.randint(0,1))

    return circuit_code


def layer(x, wires, curcuit_code):
    """Building block of the embedding ansatz"""
    for j, wire in enumerate(wires):
        for i in range(np.shape(curcuit_code)[1]):
            if i == 0:
                if(curcuit_code[j,i] == 0):
                    qml.RX(x[j], wires=[wire])
                elif(curcuit_code[j,i] == 1):
                    qml.RY(x[j], wires=[wire])
                elif(curcuit_code[j,i] == 2):
                    qml.RZ(x[j], wires=[wire])
            elif i == 1:
                if(curcuit_code[j,i] == 1):
                    qml.Hadamard(wires=[wire])
            elif i == 2:
                if(curcuit_code[j,i] == 1):
                    if j == len(wires) - 1:
                        qml.CNOT(wires = [j, 0])
                    else:
                        qml.CNOT(wires = [j, j+1])

In [8]:
def ansatz(x, curcuit_code, wires):
    """The embedding ansatz"""
    layer(x, wires, curcuit_code)


adjoint_ansatz = qml.adjoint(ansatz)


@qml.qnode(dev_kernel)
def kernel_circuit(x1, x2, curcuit_code):
    """The quantum kernel."""
    ansatz(x1, curcuit_code, wires=range(n_qubits))
    adjoint_ansatz(x2, curcuit_code, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))


def kernel_matrix(A, B, curcuit_code):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel_circuit(a, b, curcuit_code) for b in B] for a in A])

def multiple_kernel(x1,x2, curcuit_code_list, kernel_weights):
    value_list = []
    final_kernel_value = 0
    for i in range(len(curcuit_code_list)):
        value_list.append(kernel_circuit(x1,x2, curcuit_code_list[i]))
        final_kernel_value += value_list[i] * kernel_weights[i]
    return final_kernel_value


def multiple_kernel_matrix(A, B, curcuit_code_list, kernel_weights):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[multiple_kernel(a, b, curcuit_code_list, kernel_weights) for b in B] for a in A])


In [9]:
dev_global = qml.device("default.qubit", wires=2*n_qubits)
@qml.qnode(dev_global)
def get_global_system_state(x, curcuit_code_1, curcuit_code_2):
    #获取两条不同线路。同一样本的量子态，供后面计算量子互信息
    ansatz(x, curcuit_code_1, wires=range(n_qubits))
    ansatz(x, curcuit_code_2, wires=range(n_qubits, 2*n_qubits))
    list_1=[i for i in range(n_qubits)]
    list_2=[i for i in range(n_qubits, 2*n_qubits)]
    return qml.mutual_info(wires0=list_1, wires1=list_2)

In [10]:
curcuit_code_list = []
#生成随机线路方案
for i in range(run_numbers):
    curcuit_code = generate_circuit_code(n_dim)
    curcuit_code_list.append(curcuit_code)

np.savetxt("circuit_code_cls3_8_" + fs_type + "_0925_50_5.txt", curcuit_code_list, fmt = '%d')

In [11]:
final_circuit_list = utils_circuit.reshape_circuit_list(curcuit_code_list, n_dim)
curcuit_code_1 = final_circuit_list[0]
curcuit_code_2 = final_circuit_list[1]
print(get_global_system_state(train_images[0], curcuit_code_1, curcuit_code_2))

2.8536313994621154e-14
